In [2]:
from transformers import  GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset, DatasetDict
import torch

/home/lisa/miniconda3/envs/tensorboard/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Load Validation Dataset

In [4]:
arxiv_dataset = (load_dataset(path="CShorten/ML-ArXiv-Papers", split='train').train_test_split(train_size=90000, test_size=10000))
arxiv_dataset = DatasetDict({
    'train': arxiv_dataset['train'],
    'validation': arxiv_dataset['test']})

abstracts = [arxiv_dataset['validation'][0]['abstract']]


Found cached dataset csv (/home/lisa/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


### Run inference

In [11]:
import os

# trained model loading
model_articles_path = './model_title_from_abstract'


research_helper_tokenizer  = GPT2Tokenizer.from_pretrained(model_articles_path)
research_helper_model = GPT2LMHeadModel.from_pretrained(model_articles_path).to(device)

# define tokens 
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'
body = '<|body|>'
additional_special_tokens = [body]

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad,'sep_token': body}

# add tokes to tokenizer
num_added_toks = research_helper_tokenizer.add_special_tokens(special_tokens_dict)

articles = {}
for abstract_raw in abstracts:
    print("Abstract:")
    print()
    print(abstract_raw.replace("\n"," "))
    abstract_input = [bos, abstract_raw.replace("\n"," "),  body]

    abstract_ids = research_helper_tokenizer.encode(abstract_input, return_tensors = 'pt').to(device)

    generated_titles = research_helper_model.generate(
        abstract_ids, 
        max_length= 18,  
        num_return_sequences= 5,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .85,
        do_sample= True,
        top_k= 125,
        early_stopping= True
    )

    for t in generated_titles:
        text = research_helper_tokenizer.decode(t, skip_special_tokens=True)
        print()
        print("Generated Title:", text)


Abstract:

  This paper investigates the influence of different acoustic features, audio-events based features and automatic speech translation based lexical features in complex emotion recognition such as curiosity. Pretrained networks, namely, AudioSet Net, VoxCeleb Net and Deep Speech Net trained extensively for different speech based applications are studied for this objective. Information from deep layers of these networks are considered as descriptors and encoded into feature vectors. Experimental results on the EmoReact dataset consisting of 8 complex emotions show the effectiveness, yielding highest F1 score of 0.85 as against the baseline of 0.69 in the literature. 

Generated Title: Identifying hidden states from data is of great importance in the prediction and management

Generated Title: Learning to recognize visual objects without having hand-crafted labels: a novel approach

Generated Title: On the Interpretability of Decision Trees for Deep Learning: A Case Study 

Gene